In [ ]:
pip install webvtt-py

# Read .vtt Recording Transcript and seperate into 4 parameters : start,end,text and speaker

In [ ]:
import webvtt
def formatData():

  start=[]
  end=[]
  text=[]
  speaker=[]
  for caption in webvtt.read('/content/Team 4 Design Sessions.vtt'):
      start.append(caption.start)
      end.append(caption.end)
      text.append(caption.text)
      speaker.append(caption.raw_text)
  return start, end, text, speaker

start, end, text, speaker = formatData()

In [ ]:
import pandas as pd

df = pd.DataFrame(list(zip(start,end,text,speaker)), columns = ['StartTime', 'EndTime',"Text","Speaker"])

# Convert original .vtt format to a tabular format

In [ ]:
def segregateSpeaker():

  listx=df['Speaker'].str.split('>', n=1, expand=True)
  df["Speaker"]=listx[0]
  df["Speaker"]=df["Speaker"].str.replace("<v ","")


segregateSpeaker()
df.head(10)

,StartTime,EndTime,Text,Speaker
0,00:00:04.367,00:00:05.317,I just started the recording.,"Tom, Rebekah L.@DMV"
1,00:00:05.327,00:00:08.670,"Now I may have pressed\ntranscript accident, b...","Tom, Rebekah L.@DMV"
2,00:00:06.207,00:00:06.917,"Oh, thanks, Rebecca.","Rosas, Steven M.@DMV"
3,00:00:08.670,00:00:09.267,start now.,"Tom, Rebekah L.@DMV"
4,00:00:09.147,00:00:09.597,Thank you.,"Arellano, Fernanda"
5,00:00:09.847,00:00:10.497,"Thank you, Rebecca.","Arellano, Fernanda"
6,00:00:12.827,00:00:13.447,Still 7.,"Arellano, Fernanda"
7,00:00:13.457,00:00:15.427,Do you think that this my?,"Arellano, Fernanda"
8,00:00:17.467,00:00:21.689,If we did manage to have them in\ndifferent se...,"Arellano, Fernanda"
9,00:00:21.689,00:00:25.457,"transaction, at least, would\nthat alleviate t...","Arellano, Fernanda"


In [ ]:
df_new = pd.DataFrame(columns=['Start Time', 'End Time', 'Text', 'Speaker'])

# Modified DataFrame

In [ ]:
def groupSpeakerText():
  index=0
  next_pointer_j=0
  start_pointer_i=0
  while start_pointer_i<len(df):
    speaker = df['Speaker'].iloc[start_pointer_i]
    txt = df['Text'].iloc[start_pointer_i]
    next_pointer_j=start_pointer_i+1
    flag = False
    while( next_pointer_j<=len(df)-1 and speaker==df['Speaker'].iloc[next_pointer_j] ):
      txt = txt + df['Text'].iloc[next_pointer_j]
      next_pointer_j=next_pointer_j+1
      # start_pointer_i=start_pointer_i+1
      flag=True
    start_time = df['StartTime'].iloc[start_pointer_i]
    end_time = df['EndTime'].iloc[next_pointer_j-1]
    merged_text = txt
    single_speaker = df['Speaker'].iloc[start_pointer_i]
    df_new.loc[index] = [start_time,end_time,merged_text,single_speaker]
    index=index+1
    if flag==True:
      start_pointer_i = next_pointer_j
    else:
      start_pointer_i=start_pointer_i+1

groupSpeakerText()
df_new.tail(10)

,Start Time,End Time,Text,Speaker
525,01:53:07.727,01:53:09.927,Thank you everyone.Thank you for your time.,"Arellano, Fernanda"
526,01:53:10.987,01:53:11.507,Thank you.,"Milnes, Kathleen F.@DMV"
527,01:53:11.087,01:53:11.637,Thank you guys.,"Hasan, Ahmedul@DMV"
528,01:53:11.397,01:53:11.907,Thanks everyone.,"Rosas, Steven M.@DMV"
529,01:53:11.907,01:53:12.487,Thank you.,"Rodriguez, Regina A.@DMV"
530,01:53:13.347,01:53:13.877,Have a good one.,"Rosas, Steven M.@DMV"
531,01:53:15.007,01:53:15.727,You too. Bye.,"Arellano, Fernanda"
532,01:53:15.377,01:53:15.967,You.,@1
533,01:53:15.407,01:53:15.587,Right.,"Saiz, Monika H.@DMV"
534,01:53:16.037,01:53:16.497,Thank you.,@1


# People speaking in the meeting.

In [ ]:
nunique_values = df_new['Speaker'].nunique()
unique_values = df_new['Speaker'].unique()
df_spkr = pd.DataFrame(columns=['Speakers'])
df_spkr['Speakers'] = df_new['Speaker'].unique()
df_spkr

,Speakers
0,"Tom, Rebekah L.@DMV"
1,"Rosas, Steven M.@DMV"
2,"Arellano, Fernanda"
3,"Rodriguez, Regina A.@DMV"
4,"Saiz, Monika H.@DMV"
5,@1
6,"Milnes, Kathleen F.@DMV"
7,"Hasan, Ahmedul@DMV"


**Shows how many times a speaker has spoken in whole time period of a meeting**

# Count of number of time a person is speaking

In [ ]:
df_spkr_count = pd.DataFrame(columns=['Speaker','Count'])
df_spkr_count['Speaker'] = df_new['Speaker'].unique()
for i in range (0,len(df_spkr_count)):
  df_spkr_count.at[i,'Count'] = df_new['Speaker'].value_counts()[unique_values[i]]

df_spkr_count

,Speaker,Count
0,"Tom, Rebekah L.@DMV",9
1,"Rosas, Steven M.@DMV",143
2,"Arellano, Fernanda",201
3,"Rodriguez, Regina A.@DMV",43
4,"Saiz, Monika H.@DMV",68
5,@1,45
6,"Milnes, Kathleen F.@DMV",21
7,"Hasan, Ahmedul@DMV",5


# Recording Duration

In [ ]:
from datetime import datetime

def calculateMeetingDuration():
  start_time = df['StartTime'].iloc[0]
  end_time = df['EndTime'].iloc[len(df)-1]
  time_format = "%H:%M:%S.%f"
  start_time = datetime.strptime(start_time,time_format)
  end_time = datetime.strptime(end_time,time_format)
  duration =  end_time-start_time
  hours, remainder = divmod(duration.seconds, 3600)
  minutes, seconds = divmod(remainder, 60)
  return f"{hours:02d}:{minutes:02d}:{seconds:02d}.{duration.microseconds:06d}"

meeting_duration = calculateMeetingDuration()
df_metng_dur = pd.DataFrame({'Recording Duration':[meeting_duration]})
df_metng_dur

,Recording Duration
0,01:53:12.130000


In [ ]:
def calculateSpeakingDuration(start_time,end_time):
  time_format = "%H:%M:%S.%f"
  start_time = datetime.strptime(start_time,time_format)
  end_time = datetime.strptime(end_time,time_format)
  duration =  end_time-start_time
  hours, remainder = divmod(duration.seconds, 3600)
  minutes, seconds = divmod(remainder, 60)
  return f"{hours:02d}:{minutes:02d}:{seconds:02d}.{duration.microseconds:06d}"


def sum_duration(spkr_duration_sum,spkr_duration):
  spkr_duration_sum = spkr_duration_sum + parse_duration(spkr_duration)
  return spkr_duration_sum

def parse_duration(time_str):
    # Define the format of the time strings
    time_format = "%H:%M:%S.%f"
    # Convert the time string to a datetime object
    dt = datetime.strptime(time_str, time_format)
    # Convert the datetime object to a timedelta object
    td = timedelta(hours=dt.hour, minutes=dt.minute, seconds=dt.second, microseconds=dt.microsecond)
    return td

def format_duration(spkr_duration_sum):
  total_seconds = int(spkr_duration_sum.total_seconds())
  microseconds = spkr_duration_sum.microseconds
  # Calculate hours, minutes, and seconds from total_seconds
  hours, remainder = divmod(total_seconds, 3600)
  minutes, seconds = divmod(remainder, 60)

  # Format the total duration as a string
  formatted_total_duration = f"{int(hours):02}:{int(minutes):02}:{int(seconds):02}.{microseconds:06}"

  return formatted_total_duration


In [ ]:
df_spkr_duration = pd.DataFrame(columns=['Speaker','Duration'])

In [ ]:
from datetime import timedelta
def participantSpeakingDuration():
  index = 0
  for i in range(0,len(df_spkr)):
    speaker = df_spkr['Speakers'].iloc[i]
    spkr_duration_sum = timedelta()
    for j in range(0, len(df_new)):
      if (speaker == df_new['Speaker'].iloc[j]):
        spkr_start_time = df_new['Start Time'].iloc[j]
        spkr_end_time = df_new['End Time'].iloc[j]
        spkr_duration = calculateSpeakingDuration(spkr_start_time,spkr_end_time)
        spkr_duration_sum = sum_duration(spkr_duration_sum, spkr_duration)

    formatted_total_duration = format_duration(spkr_duration_sum)
    df_spkr_duration.loc[index] = [speaker,formatted_total_duration]
    index = index+1
participantSpeakingDuration()
df_spkr_duration

,Speaker,Duration
0,"Tom, Rebekah L.@DMV",00:00:18.590000
1,"Rosas, Steven M.@DMV",00:22:57.290000
2,"Arellano, Fernanda",00:46:51.150000
3,"Rodriguez, Regina A.@DMV",00:06:25.750000
4,"Saiz, Monika H.@DMV",00:14:55.610000
5,@1,00:09:43.030000
6,"Milnes, Kathleen F.@DMV",00:01:50.060000
7,"Hasan, Ahmedul@DMV",00:00:17.410000


In [ ]:
sum = timedelta()
for i in range(0,len(df_spkr_duration)):
  sum = sum + parse_duration(df_spkr_duration['Duration'].iloc[i])

duration = format_duration(sum)
duration

'01:43:18.890000'